# Preparation, protonation, building
## Toni Giorgino
Institute of Neurosciences  
National Research Council of Italy


# Abstract

This session will cover the steps **preliminary** to a simulation -- from a raw PDB file, to a set of files constituting a **runnable** system. The **system preparation** phase, based on the PDB2PQR and propKa softwares, addresses e.g. the problems of assigning  titration states at the user-chosen pH; flipping the side chains of HIS, ASN, and GLN residues; and optimizing the overall hydrogen bonding network. The **build** phase takes a prepared system and applies the chosen forcefield in order to obtain simulation-ready input files. This session provides an overview of the options available and feedback obtained during the preparation and building phases.

# Overview

This session will cover the steps **preliminary** to 
a simulation -- from a raw PDB file, to a set of
files constituting a **runnable** system.

Currently supported output formats:
*CHARMM* and *AMBER*.

We shall also deal with transmembrane domains.

<img src="img/overview.svg" style="width: 70%"/>

# Let's start

In [1]:
# %qtconsole
from htmd import *
!mkdir 03a_out_report


Please cite -- HTMD: High-Throughput Molecular Dynamics for Molecular Discovery
J. Chem. Theory Comput., 2016, 12 (4), pp 1845-1852. 
http://pubs.acs.org/doi/abs/10.1021/acs.jctc.6b00049

You are on the latest HTMD version (1.5.3).



# Part 1. Protein preparation

The system preparation phase is based on the PDB2PQR software. It 
includes the following steps (from the
[PDB2PQR algorithm
description](http://www.poissonboltzmann.org/docs/pdb2pqr-algorithm-description/)):

 * Compute empirical pKa values for the residues' local environment (propKa)
 * Assign titration states at the user-chosen pH;
 * Flipping the side chains of HIS (including user defined HIS states), ASN, and GLN residues;
 * Rotating the sidechain hydrogen on SER, THR, TYR, and CYS (if available);
 * Determining the best placement for the sidechain hydrogen on neutral HIS, protonated GLU, and protonated ASP;
 * Optimizing all water hydrogens.

The hydrogen bonding network calculations are performed by the
[PDB2PQR](http://www.poissonboltzmann.org/) software package. The pKa
calculations are performed by the [PROPKA
3.1](https://github.com/jensengroup/propka-3.1) software packages.
Please see the copyright, license  and citation terms distributed with each.

Note that this version was modified in order to use an 
externally-supplied propKa **3.1** (installed automatically via dependencies), whereas
the original had propKa 3.0 *embedded*!

The results of the function should be roughly equivalent of the system
preparation wizard's preprocessing and optimization steps
of Schrodinger's Maestro software.

<img src="img/naming.svg" style="width: 70%"/>

Modified residue names
----------------------

The molecule produced by the preparation modifies residue names
according to their protonation.
Later system-building functions assume these residue naming conventions. 
**Note**: support for alternative charge states varies between the  forcefields.

Charge +1    |  Neutral   | Charge -1
-------------|------------|----------
 -           |  ASH       | ASP
 -           |  CYS       | CYM
 -           |  GLH       | GLU
HIP          |  HID/HIE   |  -
LYS          |  LYN       |  -
 -           |  TYR       | TYM
ARG          |  AR0       |  -



The `proteinPrepare` function requires a `Molecule` object, the protein to be prepared, as an argument, and returns the prepared system, also as a `Molecule`. Logging messages will provide information and warnings about the process.

```python
def proteinPrepare(mol_in,
                   pH=7.0,
                   verbose=0,
                   returnDetails=False,
                   hydrophobicThickness=None,
                   holdSelection=None):
```

Returns a Molecule object, where residues have been renamed to follow
internal conventions on protonation (below). Coordinates are changed to
optimize the H-bonding network. This should be roughly comparable to
Schroedinger Maestro's preparation wizard.

## Parameters

    mol_in : htmd.Molecule
        the object to be optimized
    pH : float
        pH to decide titration
    verbose : int
        verbosity
    returnDetails : bool
        whether to return just the prepared Molecule (False, default) or a molecule *and* a ResidueInfo
        object including computed properties
    hydrophobicThickness : float
        the thickness of the membrane in which the protein is embedded, or None if globular protein.
        Used to provide a warning about membrane-exposed residues.
    holdSelection : str
        Atom selection to be excluded from optimization.
        Only the carbon-alpha atom will be considered for the corresponding residue.

`proteinPrepare()` is a convenience function. Using it
is **not** mandatory. You can 
manipulate the input molecule with your custom functions. 
In particular,

* Addition of hydrogen atoms is not required
* Protonation states are set by renaming residues
* HIS and other residues can be edited as coordinates



## Example

Prepare trypsin (PDB: 3PTB) at pH 7.

In [2]:
tryp = Molecule("3PTB")
tryp_op = proteinPrepare(tryp)

2016-11-09 11:20:16,909 - htmd.molecule.molecule - INFO - Using local copy for 3PTB: /home/ec2-user/miniconda3/lib/python3.5/site-packages/htmd/data/pdb/3ptb.pdb
2016-11-09 11:20:17,169 - propka - INFO - No pdbfile provided
2016-11-09 11:20:19,951 - htmd.builder.preparation - WARNING - The following residue has not been optimized: CA
2016-11-09 11:20:19,952 - htmd.builder.preparation - WARNING - The following residue has not been optimized: BEN
2016-11-09 11:20:24,451 - htmd.builder.residuedata - INFO - The following residues are in a non-standard state: CYS    22  A (CYX), HIS    40  A (HIE), CYS    42  A (CYX), HIS    57  A (HIP), CYS    58  A (CYX), HIS    91  A (HID), CYS   128  A (CYX), CYS   136  A (CYX), CYS   157  A (CYX), CYS   168  A (CYX), CYS   182  A (CYX), CYS   191  A (CYX), CYS   201  A (CYX), CYS   220  A (CYX), CYS   232  A (CYX)
2016-11-09 11:20:24,453 - htmd.builder.residuedata - WARNING - Dubious protonation state: the pKa of 3 residues is within 1.0 units of pH 7.

## Preparation report

If the `returnDetails` argument is set,  an object of type `ResidueData` is returned as a **second** return value. It carries a wealth of information on the preparation results. 

In [3]:
tryp_op, prepData = proteinPrepare(tryp, returnDetails=True)
prepData

2016-11-09 11:20:27,326 - htmd.builder.preparation - WARNING - The following residue has not been optimized: CA
2016-11-09 11:20:27,327 - htmd.builder.preparation - WARNING - The following residue has not been optimized: BEN
2016-11-09 11:20:31,773 - htmd.builder.residuedata - INFO - The following residues are in a non-standard state: CYS    22  A (CYX), HIS    40  A (HIE), CYS    42  A (CYX), HIS    57  A (HIP), CYS    58  A (CYX), HIS    91  A (HID), CYS   128  A (CYX), CYS   136  A (CYX), CYS   157  A (CYX), CYS   168  A (CYX), CYS   182  A (CYX), CYS   191  A (CYX), CYS   201  A (CYX), CYS   220  A (CYX), CYS   232  A (CYX)
2016-11-09 11:20:31,775 - htmd.builder.residuedata - WARNING - Dubious protonation state: the pKa of 3 residues is within 1.0 units of pH 7.0.
2016-11-09 11:20:31,777 - htmd.builder.residuedata - WARNING - Dubious protonation state:    HIS    57  A (pKa= 7.44)
2016-11-09 11:20:31,778 - htmd.builder.residuedata - WARNING - Dubious protonation state:    GLU    70 

ResidueData object about 290 residues.
Unparametrized residue names: CA, BEN
Please find the full info in the .data property, e.g.: 
  resname  resid insertion chain       pKa protonation flipped     buried
0     ILE     16               A       NaN         ILE     NaN        NaN
1     VAL     17               A       NaN         VAL     NaN        NaN
2     GLY     18               A       NaN         GLY     NaN        NaN
3     GLY     19               A       NaN         GLY     NaN        NaN
4     TYR     20               A  9.590845         TYR     NaN  14.642857
 . . .

Most of it is accessible in the `data` property (a pandas `DataFrame`).

In [4]:
prepData.data

,resname,resid,insertion,chain,pKa,protonation,flipped,patches,buried,z,membraneExposed,pka_group_id,pka_residue_type,pka_type,pka_charge,pka_atom_name,pka_atom_sybyl_type
0,ILE,16,,A,NaN,ILE,NaN,[NTERM],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VAL,17,,A,NaN,VAL,NaN,[PEPTIDE],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GLY,18,,A,NaN,GLY,NaN,[PEPTIDE],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GLY,19,,A,NaN,GLY,NaN,[PEPTIDE],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TYR,20,,A,9.590845,TYR,NaN,[PEPTIDE],14.642857,18.435,NaN,1.0,TYR,TYR,-1,OH,
5,THR,21,,A,NaN,THR,NaN,[PEPTIDE],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,CYS,22,,A,99.990000,CYX,NaN,"[PEPTIDE, CYX]",0.000000,23.800,NaN,2.0,CYS,CYS,-1,SG,
7,GLY,23,,A,NaN,GLY,NaN,[PEPTIDE],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ALA,24,,A,NaN,ALA,NaN,[PEPTIDE],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ASN,25,,A,NaN,ASN,True,[PEPTIDE],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As such, it can be easily queried and written as a spreadsheet in Excel or CSV format.

In [5]:
prepData.data.to_excel("./03a_out_report/tryp_data.xlsx")

## Membrane proteins

Membrane-embedded proteins are in contact with an hydrophobic region which may alter pKa values for membrane-exposed residues ([Teixera et al., J. Chem. Theory Comput., 2016, 12 (3), pp 930–934](http://dx.doi.org/10.1021/acs.jctc.5b01114)). Although the effect is not currently   taken into account quantitatively, if a `hydrophobicThickness` argument is provided, warnings will be generated for residues exposed to the lipid region.

<img src="img/ct-2015-01114c_0002.jpeg" style="width: 70%"/>
<!-- http://pubs.acs.org/appl/literatum/publisher/achs/journals/content/jctcce/2016/jctcce.2016.12.issue-3/acs.jctc.5b01114/20160302/images/large/ct-2015-01114c_0002.jpeg -->

Residue pKa values along the membrane normal. Negative insertion values correspond to deeper membrane insertions, while positive values correspond to more shallow locations. The insertion values were measured between the titrable group and the phosphate from the nearest lipid (see Methods and Supporting Information for details). The aqueous bulk pKa values of the pentapeptides are shown on top for comparison. Ctr and Ntr correspond to the C- and N-terminus, respectively. The two horizontal lines at ∼1 Å and ∼−6 Å correspond to the average positions of the choline nitrogens and the second carbon atoms of the acyl chains, respectively.

The following example shows the preparation of the mu opioid receptor, 4DKL. 
The **pre-oriented** structure is retrieved  from the OPM database.

In [6]:
mor, thickness = htmd.util.opm("4dkl") 
print(thickness)
mor.filter("protein and noh")
mor_opt, mor_data = proteinPrepare(mor, returnDetails=True,
                                   hydrophobicThickness=thickness)

exposedRes = mor_data.data.membraneExposed
mor_data.data[exposedRes]
mor_data.data[exposedRes].to_excel("./03a_out_report/mor_exposed_residues.xlsx")

2016-11-09 11:20:34,534 - htmd.molecule.molecule - INFO - Removed 2546 atoms. 4836 atoms remaining in the molecule.
2016-11-09 11:20:34,592 - htmd.molecule.molecule - INFO - Removed 364 atoms. 4472 atoms remaining in the molecule.


32.0


2016-11-09 11:20:52,510 - htmd.builder.residuedata - INFO - The following residues are in a non-standard state: ASP   114  A (ASH), CYS   140  A (CYX), HIS   171  A (HID), CYS   217  A (CYX), HIS   223  A (HID), HIS   297  A (HID), HIS   319  A (HIE), ASP   114  B (ASH), CYS   140  B (CYX), HIS   171  B (HID), CYS   217  B (CYX), HIS   223  B (HID), HIS   297  B (HID), HIS   319  B (HIE)
2016-11-09 11:20:52,513 - htmd.builder.residuedata - WARNING - Dubious protonation state: the pKa of 6 residues is within 1.0 units of pH 7.0.
2016-11-09 11:20:52,514 - htmd.builder.residuedata - WARNING - Dubious protonation state:    ASP   114  A (pKa= 7.85)
2016-11-09 11:20:52,515 - htmd.builder.residuedata - WARNING - Dubious protonation state:    HIS   223  A (pKa= 6.36)
2016-11-09 11:20:52,516 - htmd.builder.residuedata - WARNING - Dubious protonation state:    ASP   114  B (pKa= 7.85)
2016-11-09 11:20:52,517 - htmd.builder.residuedata - WARNING - Dubious protonation state:    HIS   223  B (pKa= 

# Step 2: Building

Only the basics - please find extensive tutorials at www.htmd.org .

In [7]:
# prot=Molecule("bentryp/trypsin.pdb")
# prot.filter('chain A and (protein or water or resname CA)')


## Case 1. Globular protein, no ligand

### Step 1a. Segment.

In [8]:
tryp = Molecule("3PTB")
tryp.remove("resname BEN")
tryp_op = proteinPrepare(tryp)
tryp_seg = autoSegment(tryp_op)

2016-11-09 11:20:52,571 - htmd.molecule.molecule - INFO - Using local copy for 3PTB: /home/ec2-user/miniconda3/lib/python3.5/site-packages/htmd/data/pdb/3ptb.pdb
2016-11-09 11:20:52,672 - htmd.molecule.molecule - INFO - Removed 9 atoms. 1692 atoms remaining in the molecule.
2016-11-09 11:20:55,556 - htmd.builder.preparation - WARNING - The following residue has not been optimized: CA
2016-11-09 11:21:00,002 - htmd.builder.residuedata - INFO - The following residues are in a non-standard state: CYS    22  A (CYX), HIS    40  A (HIE), CYS    42  A (CYX), HIS    57  A (HIP), CYS    58  A (CYX), HIS    91  A (HID), CYS   128  A (CYX), CYS   136  A (CYX), CYS   157  A (CYX), CYS   168  A (CYX), CYS   182  A (CYX), CYS   191  A (CYX), CYS   201  A (CYX), CYS   220  A (CYX), CYS   232  A (CYX)
2016-11-09 11:21:00,004 - htmd.builder.residuedata - WARNING - Dubious protonation state: the pKa of 4 residues is within 1.0 units of pH 7.0.
2016-11-09 11:21:00,006 - htmd.builder.residuedata - WARNIN

### Step 1b. Solvate

In [9]:
tryp_solv = solvate(tryp_seg,pad=5)
# tryp_solv.view()

2016-11-09 11:21:01,183 - htmd.builder.solvate - INFO - Using water pdb file at: /home/ec2-user/miniconda3/lib/python3.5/site-packages/htmd/builder/wat.pdb
2016-11-09 11:21:02,112 - htmd.builder.solvate - INFO - Replicating 1 water segments, 1 by 1 by 1


Solvating: 100% (1/1) [############################################] eta --:-- -

2016-11-09 11:21:03,853 - htmd.builder.solvate - INFO - After removing water molecules colliding with other molecules, 3552 water molecules were added to the system.


### Step 1c. Build (+ionize) for CHARMM.

This step also ionizes the system (option `saltconc`).

In [10]:
topos  = ['top/top_all22star_prot.rtf']
params = ['par/par_all22star_prot.prm']

tryp_charmm = charmm.build(tryp_solv, topo=topos, param=params, 
                           outdir='03a_out_amber')

2016-11-09 11:21:06,090 - htmd.builder.charmm - INFO - Writing out segments.


Bond between A: [serial 92 resid 22 resname CYX chain A segid P0]
             B: [serial 1988 resid 157 resname CYX chain A segid P0]

Bond between A: [serial 351 resid 42 resname CYX chain A segid P0]
             B: [serial 571 resid 58 resname CYX chain A segid P0]

Bond between A: [serial 1604 resid 128 resname CYX chain A segid P0]
             B: [serial 3004 resid 232 resname CYX chain A segid P0]

Bond between A: [serial 1683 resid 136 resname CYX chain A segid P0]
             B: [serial 2611 resid 201 resname CYX chain A segid P0]

Bond between A: [serial 2146 resid 168 resname CYX chain A segid P0]
             B: [serial 2353 resid 182 resname CYX chain A segid P0]

Bond between A: [serial 2494 resid 191 resname CYX chain A segid P0]
             B: [serial 2799 resid 220 resname CYX chain A segid P0]



2016-11-09 11:21:22,658 - htmd.builder.builder - INFO - 6 disulfide bonds were added
2016-11-09 11:21:22,763 - htmd.builder.charmm - INFO - Starting the build.
2016-11-09 11:21:22,903 - htmd.builder.charmm - WARNING - Failed to set coordinates for 8 atoms.
2016-11-09 11:21:22,904 - htmd.builder.charmm - WARNING - Failed to guess coordinates for 2 atoms due to bad angles.
2016-11-09 11:21:22,904 - htmd.builder.charmm - WARNING - Poorly guessed coordinates for 15 atoms.
2016-11-09 11:21:22,905 - htmd.builder.charmm - WARNING - Please check /home/ec2-user/03_PreparationBuilding_TGiorgino/03a_out_amber/log.txt for further information.
2016-11-09 11:21:22,905 - htmd.builder.charmm - INFO - Finished building.
2016-11-09 11:21:23,788 - htmd.builder.ionize - INFO - Adding 9 anions + 0 cations for neutralizing and 0 ions for the given salt concentration.
2016-11-09 11:21:23,898 - htmd.builder.ionize - INFO - Min distance of ions from molecule: 5A
2016-11-09 11:21:23,898 - htmd.builder.ionize - 

### Step 1c (alternative). Build for AMBER.

This function also ionizes the system (option `saltconc`).

(TIP3P parameters for Ca++ are required - using `frcmod.ionslrcm_cm_tip3p`. See [link](https://github.com/pandegroup/openmm/issues/726) )

In [11]:
params = ["frcmod.ionslrcm_cm_tip3p"]
tryp_amber = amber.build(tryp_solv, param=params, outdir='./03a_out_amber')

2016-11-09 11:21:45,086 - htmd.builder.amber - INFO - Converting CHARMM membranes to AMBER.
2016-11-09 11:21:46,656 - htmd.builder.amber - INFO - Writing PDB file for input to tleap.
2016-11-09 11:21:47,863 - htmd.builder.amber - INFO - Starting the build.
2016-11-09 11:21:49,215 - htmd.builder.amber - INFO - Finished building.
2016-11-09 11:21:49,865 - htmd.builder.ionize - INFO - Adding 9 anions + 0 cations for neutralizing and 0 ions for the given salt concentration.
2016-11-09 11:21:49,975 - htmd.builder.ionize - INFO - Min distance of ions from molecule: 5A
2016-11-09 11:21:49,976 - htmd.builder.ionize - INFO - Min distance between ions: 5A
2016-11-09 11:21:49,976 - htmd.builder.ionize - INFO - Placing 9 ions.
2016-11-09 11:21:52,040 - htmd.builder.builder - WARNING - Segments ['P0'] contain both protein and non-protein atoms. Please assign separate segments to them or the build procedue might fail.
2016-11-09 11:21:52,041 - htmd.builder.amber - INFO - Converting CHARMM membranes 

Bond between A: [serial 90 resid 22 resname CYX chain A segid P0]
             B: [serial 1986 resid 157 resname CYX chain A segid P0]

Bond between A: [serial 349 resid 42 resname CYX chain A segid P0]
             B: [serial 569 resid 58 resname CYX chain A segid P0]

Bond between A: [serial 1602 resid 128 resname CYX chain A segid P0]
             B: [serial 3002 resid 232 resname CYX chain A segid P0]

Bond between A: [serial 1681 resid 136 resname CYX chain A segid P0]
             B: [serial 2609 resid 201 resname CYX chain A segid P0]

Bond between A: [serial 2144 resid 168 resname CYX chain A segid P0]
             B: [serial 2351 resid 182 resname CYX chain A segid P0]

Bond between A: [serial 2492 resid 191 resname CYX chain A segid P0]
             B: [serial 2797 resid 220 resname CYX chain A segid P0]



2016-11-09 11:21:54,265 - htmd.builder.builder - INFO - 6 disulfide bonds were added


KeyboardInterrupt: 

# Output

The output is both a `Molecule` object, and files in the output directory specified. 
These are topologies needed by the simulation software.

In [12]:
!ls ./03a_out_charmm

ls: cannot access ./03a_out_charmm: No such file or directory


In [13]:
!ls ./03a_out_amber

0.top_all22star_prot.rtf   segmentP10.pdb  segmentP18.pdb  segmentP4.pdb
1.top_all36_prot_arg0.rtf  segmentP11.pdb  segmentP19.pdb  segmentP5.pdb
build.vmd		   segmentP12.pdb  segmentP1.pdb   segmentP6.pdb
frcmod.ionslrcm_cm_tip3p   segmentP13.pdb  segmentP20.pdb  segmentP7.pdb
input.pdb		   segmentP14.pdb  segmentP21.pdb  segmentP8.pdb
parameters		   segmentP15.pdb  segmentP22.pdb  segmentP9.pdb
segmentI.pdb		   segmentP16.pdb  segmentP2.pdb   segmentWT0.pdb
segmentP0.pdb		   segmentP17.pdb  segmentP3.pdb   tleap.in


## Case 2. Building with a ligand

Coexistence and automatic placement of a ligand requires further manipulation,
because:

1. The ligand may have to be arranged in a geometrically sensible way
2. We likely need additional parameters and topologies (M. J. Harvey's parametrization session)

See the tutorial [System Building Trypsin-Benzamidine](https://www.htmd.org/docs/latest/tutorials/system-building-protein-ligand.html).

## Case 3. Membrane proteins

Pre-equilibrated membranes can be merged with pre-oriented systems,
e.g. downloaded from the OPM. See the tutorial [System Building μ-opioid Receptor in Membrane](https://www.htmd.org/docs/latest/tutorials/system-building-protein-in-membrane.html).

# Citations

Please acknowledge your use of PDB2PQR by citing...
 *   Dolinsky TJ, Czodrowski P, Li H, Nielsen JE, Jensen JH, Klebe G, Baker NA. PDB2PQR: Expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res, 35, W522-5, 2007. 

For propKa...
 *   Sondergaard, Chresten R., Mats HM Olsson, Michal Rostkowski, and Jan H. Jensen. "Improved Treatment of Ligands and Coupling Effects in Empirical Calculation and Rationalization of pKa Values." Journal of Chemical Theory and Computation 7, no. 7 (2011): 2284-2295.
